# How To: Run SModelS as a python library

In [1]:
# Set up the path to SModelS installation folder
import sys; sys.path.append("."); import smodels_paths

In [2]:
from imp import reload
from smodels.tools import runtime
from smodels import particlesLoader
from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList                                                   
from smodels.share.models.SMparticles import SMList                                           
from smodels.theory.model import Model
setLogLevel("info")

### Main input:

In [3]:
#Define your model (list of rEven and rOdd particles)
runtime.modelFile = 'smodels.share.models.mssm' 
model = Model(BSMparticles=BSMList, SMparticles=SMList)
 
slhafile = 'inputFiles/slha/lightEWinos.slha'
model.updateParticles ( inputFile = slhafile )

### Decompose the input model:

In [4]:
# Set main options for decomposition
sigmacut = 0.01 * fb
mingap = 5. * GeV
# Decompose model (use slhaDecomposer for SLHA input or lheDecomposer for LHE input)
toplist = decomposer.decompose(model, sigmacut, doCompress=True, doInvisible=True, minmassgap=mingap)

# Access basic information from decomposition, using the topology list and topology objects:
print( "\n Decomposition Results: " )
print( "\t  Total number of topologies: %i " %len(toplist) )
nel = sum([len(top.elementList) for top in toplist])
print( "\t  Total number of elements = %i " %nel )


 Decomposition Results: 
	  Total number of topologies: 51 
	  Total number of elements = 14985 


### Load the Database of experimental results:

In [5]:
# Set the path to the database
database = Database("official")
# Load the experimental results to be used.
# In this case, all results are employed.
listOfExpRes = database.getExpResults()

# Print basic information about the results loaded.
# Count the number of loaded UL and EM experimental results:
nUL, nEM = 0, 0
for exp in listOfExpRes:
    expType = exp.getValuesFor('dataType')[0]
    if expType == 'upperLimit':
        nUL += 1
    elif  expType == 'efficiencyMap':
        nEM += 1
print("\n Loaded Database with %i UL results and %i EM results " %(nUL,nEM))

INFO in databaseObj.loadBinaryFile() in 222: loading binary db file /home/walten/.cache/smodels/official200-beta.pcl format version 213
INFO in databaseObj.loadBinaryFile() in 229: Loaded database from /home/walten/.cache/smodels/official200-beta.pcl in 1.2 secs.



 Loaded Database with 84 UL results and 30 EM results 


### Match the decomposed simplified models with the experimental database of constraints:

In [9]:
# Compute the theory predictions for each experimental result and print them:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, toplist)
    if not predictions: continue # Skip if there are no constraints from this result
    print('\n %s (%i TeV)' %(expResult.globalInfo.id,expResult.globalInfo.sqrts.asNumber(TeV)))
    for theoryPrediction in predictions:
        dataset = theoryPrediction.dataset
        datasetID = theoryPrediction.dataId()
        mass = theoryPrediction.mass
        txnames = [str(txname) for txname in theoryPrediction.txnames]
        PIDs =  theoryPrediction.PIDs         
        print( "------------------------" )
        print( "TxNames = ",txnames )  
        print( "Theory Prediction = ",theoryPrediction.xsection.value )  #Signal cross section
        # Get the corresponding upper limit:
        print( "UL for theory prediction = ",theoryPrediction.upperLimit )
        # Compute the r-value
        r = theoryPrediction.getRValue()
        print( "r = ",r )
        #Compute likelihhod and chi^2 for EM-type results:
        if theoryPrediction.dataType() == 'efficiencyMap':
            theoryPrediction.computeStatistics()
            print( 'Chi2, likelihood=', theoryPrediction.chi2, theoryPrediction.likelihood )
        if r > rmax:
            rmax = r
            bestResult = expResult.globalInfo.id

# Print the most constraining experimental result
print( "\nThe largest r-value (theory/upper limit ratio) is ",rmax )
if rmax > 1.:
    print( "(The input model is likely excluded by %s)" %bestResult )
else:
    print( "(The input model is not excluded by the simplified model results)" )


 Theory Predictions and Constraints:

 ATLAS-SUSY-2015-06 (13 TeV)
------------------------
TxNames =  ['T1', 'T2']
Theory Prediction =  5.28E-06 [pb]
UL for theory prediction =  1.79E+00 [fb]
r =  0.00295062967537918
Chi2, likelihood= 2.6537778747865906 0.007168383173308842

 ATLAS-SUSY-2016-07 (13 TeV)
------------------------
TxNames =  ['T2']
Theory Prediction =  2.54E-05 [pb]
UL for theory prediction =  3.83E+02 [fb]
r =  6.620539967185558e-05

 ATLAS-SUSY-2016-07 (13 TeV)
------------------------
TxNames =  ['TGQ', 'T1', 'T2', 'T5WWoff']
Theory Prediction =  1.25E-04 [pb]
UL for theory prediction =  1.27E+00 [fb]
r =  0.09829040889022296
Chi2, likelihood= 0.018825265630777466 0.0009686553349033912

 ATLAS-SUSY-2016-24 (13 TeV)
------------------------
TxNames =  ['TSlepSlep']
Theory Prediction =  2.24E-05 [pb]
UL for theory prediction =  2.25E+00 [fb]
r =  0.009984620504027271
------------------------
TxNames =  ['TChiWZ']
Theory Prediction =  1.61E-03 [pb]
UL for theory predict


 CMS-PAS-SUS-13-016 (8 TeV)
------------------------
TxNames =  ['T1tttt']
Theory Prediction =  6.70E-04 [pb]
UL for theory prediction =  9.10E+01 [fb]
r =  0.007368624373644708

 CMS-PAS-SUS-13-016 (8 TeV)
------------------------
TxNames =  ['T1tttt']
Theory Prediction =  2.55E-06 [pb]
UL for theory prediction =  2.11E-01 [fb]
r =  0.01208968260427226
Chi2, likelihood= 0.02869660826577647 0.1379640891331589

 CMS-SUS-12-024 (8 TeV)
------------------------
TxNames =  ['T1tttt']
Theory Prediction =  6.70E-04 [pb]
UL for theory prediction =  2.03E+02 [fb]
r =  0.003303978838246602

 CMS-SUS-12-024 (8 TeV)
------------------------
TxNames =  ['T1bbbb']
Theory Prediction =  1.65E-04 [pb]
UL for theory prediction =  4.25E-01 [fb]
r =  0.38708893994109456
Chi2, likelihood= 0.919445700433533 0.016744770181117058

 CMS-SUS-12-028 (8 TeV)
------------------------
TxNames =  ['T1']
Theory Prediction =  2.03E-04 [pb]
UL for theory prediction =  1.28E+02 [fb]
r =  0.0015825004905329375
--------

### Check for simplified models in the input model which were not tested by the Database:

In [13]:
#Find out missing topologies for sqrts=8*TeV:
uncovered = coverage.Uncovered(toplist,sqrts=8.*TeV)
#Print uncovered cross-sections:
print( "\nTotal missing topology cross section (fb): %10.3E\n" %(uncovered.getTotalXSec()) )
print( "Total cross section where we are outside the mass grid (fb): %10.3E\n" %(uncovered.getOutOfGridXsec()) )
print( "Total cross section in long cascade decays (fb): %10.3E\n" %(uncovered.getLongCascadeXsec()) )
print( "Total cross section in decays with asymmetric branches (fb): %10.3E\n" %(uncovered.getAsymmetricXsec()) )

#Print some of the missing topologies:
print( 'Missing topologies (up to 3):' )
for topo in uncovered.missingTopos.topos[:3]:
    print( 'Topology:',topo.topo )
    print( 'Contributing elements (up to 2):' )
    for el in topo.contributingElements[:2]:
        print( el,'cross-section (fb):', el.missingX )

#Print elements with long cascade decay:
print( '\nElements outside the grid (up to 2):' )
for topo in uncovered.outsideGrid.topos[:2]:
    print( 'Topology:',topo.topo )
    print( 'Contributing elements (up to 4):' )
    for el in topo.contributingElements[:4]:
        print( el,'cross-section (fb):', el.missingX )
        print( '\tmass:',el.getMasses() )


AttributeError: 'Uncovered' object has no attribute 'getOutOfGridXsec'